#### Clickhouse

В Clickhouse мы создаем 5 таблиц/view.

In [4]:
from clickhouse_driver import Client
import secret
import warnings
warnings.filterwarnings('ignore')

In [5]:
client = Client(host=secret.host, port=9000, user=secret.user, password=secret.password, database='hardda')

Таблица с движком S3, которая смотрит на сформированный ранее плоский файл (Step 1). Важно! Необходимо верно указать типы и названия полей.

In [37]:
create_1 = """CREATE TABLE flat_table (
            execution_date TIMESTAMP,
            advert_id UInt64,
            region VARCHAR(20),
            user_id UInt64,
            platform VARCHAR(20),
            mark VARCHAR(20),
            model VARCHAR(20),
            price Float64,
            year Float64,
            fuel VARCHAR(20),
            color VARCHAR(20),
            transmission VARCHAR(20),
            body VARCHAR(20),
            country VARCHAR(20),
            global_id Float64,
            user_type_name VARCHAR(20)
            ) 
            ENGINE=S3('https://storage.yandexcloud.net/kc-hardda-projects/student_name/flat_table', 'parquet')
            SETTINGS input_format_with_names_use_header = 0
"""

query_1 = 'SELECT * FROM flat_table LIMIT 5'

In [ ]:
client.execute(create_1)
client.execute(query_1, with_column_types=True)

In [41]:
client.execute(create_1)
result, columns = client.execute(query_1, with_column_types=True)
# df = pd.DataFrame(result, columns=[tuple[0] for tuple in columns])
# df.head()

Таблица mergeTree, куда мы зальем все наши данные из таблицы с движком S3.

In [42]:
create_2 = """CREATE TABLE mergeTree_table (
            execution_date TIMESTAMP,
            advert_id UInt64,
            region VARCHAR(20),
            user_id UInt64,
            platform VARCHAR(20),
            mark VARCHAR(20),
            model VARCHAR(20),
            price Float64,
            year Float64,
            fuel VARCHAR(20),
            color VARCHAR(20),
            transmission VARCHAR(20),
            body VARCHAR(20),
            country VARCHAR(20),
            global_id Float64,
            user_type_name VARCHAR(20)
            ) 
            ENGINE = MergeTree()
            ORDER BY (advert_id, execution_date)
            PARTITION BY toMonday(execution_date)
"""

insert = """INSERT INTO hardda.mergeTree_table (*) SELECT * from hardda.flat_table"""

query_2 = 'SELECT * FROM mergeTree_table LIMIT 5'

In [ ]:
client.execute(create_2)
client.execute(insert)
client.execute(query_2, with_column_types=True)

#### MATERIALIZED VIEW 
для решения задачи бизн
группировка по:
- execution_date,
- platform,
- user_type_name,
Расчетные поля:
- минимальная стоимость
- 0.25 квантиль
- медиана
- среднее
- 0.75 квантиль
- максимальная стоимость

In [43]:
create_3 = """CREATE TABLE hardda.agg_view_1 (
                'execution_date' TIMESTAMP,
                'platform' VARCHAR(20),
                'user_type_name' VARCHAR(20),
                'min_price' AggregateFunction(min, Float64),
                '25_price' AggregateFunction(quantile(0.25), Float64),
                'median_price' AggregateFunction(median, Float64),
                'mean_price' AggregateFunction(avg, Float64),
                '75_price' AggregateFunction(quantile(0.75), Float64),
                'max_price' AggregateFunction(max, Float64)
            )
            ENGINE = AggregatingMergeTree
            ORDER BY (execution_date)"""


agg_view_1 = """CREATE MATERIALIZED VIEW hardda.agg_view_1_mv
                TO hardda.agg_view_1
                AS
                SELECT
                    execution_date,
                    platform,
                    user_type_name,
                    min(price) as min_price,
                    quantile(0.25)(price) as 25_price,
                    median(price) as median_price,
                    avg(price) as mean_price,
                    quantile(0.75)(price) as 75_price,
                    max(price) as max_price
                FROM hardda.mergeTree_table
                GROUP BY
                    execution_date,
                    platform,
                    user_type_name"""

In [ ]:
client.execute(create_3)
client.execute(agg_view_1)